## SILVER TO GOLD LAYER

### Gold Layer - Trade History


In [1]:
# Import necessary libraries and utility functions
import pandas as pd
from common_utilities import Portfolio, global_path, logger

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [2]:
# Read the CSV file
df_trade_history = pd.read_csv(global_path.tradehistory_silver_file_path)

# Convert 'datetime' to datetime type
df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

# Sort the DataFrame by 'datetime'
df_trade_history = df_trade_history.sort_values(by="datetime")

logger.info(
    f"Read SILVER Layer trade history data from: {global_path.tradehistory_silver_file_path}"
)

2024-08-02T19:10:00Z - INFO - Read SILVER Layer trade history data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\TradeHistory\TradeHistory_data.csv


### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks and round necessary columns.


In [9]:
from pydantic import BaseModel, field_validator
from datetime import datetime
from typing import Optional, Union, Dict, List
from datetime import datetime
from typing import Any, Dict, List, Optional, Union
from pydantic import BaseModel, field_validator


class TradeRecord(BaseModel):
    datetime: datetime
    exchange: Optional[str]
    segment: Optional[str]
    stock_name: str
    side: str
    amount: Union[float, int]
    quantity: Union[float, int]
    price: Union[float, int]
    expiry_date: Optional[datetime]

    @field_validator("expiry_date", mode="before")
    def parse_expiry_date(cls, value):
        try:
            return (
                None
                if str(value) in (None, "nan", "")
                else datetime.strptime(str(value), "%Y-%m-%d")
            )
        except ValueError as e:
            raise ValueError(f"Invalid expiry date format: {e}")


class Stock(BaseModel):
    stock_name: str
    expiry_date: Optional[datetime]
    holding_quantity: float = 0.0
    avg_price: float = 0.0
    holding_amount: float = 0.0
    pnl_amount: float = 0.0
    pnl_percentage: float = 0.0
    trade_history: List[TradeRecord] = []

    def trade(self, trade_record: TradeRecord):
        # Update the stock with the trade information
        if trade_record.side == "BUY":
            total_cost = self.holding_amount + (trade_record.quantity * trade_record.price)
            self.holding_quantity += trade_record.quantity
            self.avg_price = total_cost / self.holding_quantity
            self.holding_amount += trade_record.amount
        elif trade_record.side == "SELL":
            self.holding_quantity -= trade_record.quantity
            self.holding_amount -= trade_record.amount
            if self.holding_quantity > 0:
                self.avg_price = self.holding_amount / self.holding_quantity
            else:
                self.avg_price = 0.0
        
        # Update the PnL based on the sale
        self.pnl_amount = (trade_record.price - self.avg_price) * trade_record.quantity
        self.pnl_percentage = (self.pnl_amount / self.holding_amount) * 100 if self.holding_amount != 0 else 0.0
        self.trade_history.append(trade_record)

    def check_expired(self):
        # Check if the stock is expired and reset holdings if necessary
        if self.expiry_date and datetime.now() > self.expiry_date:
            print(f"Stock {self.stock_name} is expired.")
            self.holding_quantity = 0.0
            self.holding_amount = 0.0
            self.avg_price = 0.0
            self.pnl_amount = 0.0
            self.pnl_percentage = 0.0


class Portfolio:
    def __init__(self):
        self.stocks: Dict[str, Stock] = {}

    def trade(self, trade_record: TradeRecord):
        if trade_record.stock_name not in self.stocks:
            self.stocks[trade_record.stock_name] = Stock(
                stock_name=trade_record.stock_name,
                expiry_date=trade_record.expiry_date,
            )
        self.stocks[trade_record.stock_name].trade(trade_record)

    def check_expired_stocks(self):
        for stock in self.stocks.values():
            stock.check_expired()

    def get_trade_history(self):
        return [stock.trade_history for stock in self.stocks.values()]


# Example usage
portfolio = Portfolio()
trade_data = {
    "datetime": "2020-04-21 14:41:30",
    "exchange": "NSE",
    "segment": "EQ",
    "stock_name": "TATAMOTORS",
    "side": "BUY",
    "amount": "1051.4",
    "quantity": "14.0",
    "price": "75.1",
    "expiry_date": "nan",
}

trade_record = TradeRecord(**trade_data)
portfolio.trade(trade_record)

# Check expired stocks
portfolio.check_expired_stocks()

# Retrieve trade history
trade_history = portfolio.get_trade_history()
print(trade_history)


[[TradeRecord(datetime=datetime.datetime(2020, 4, 21, 14, 41, 30), exchange='NSE', segment='EQ', stock_name='TATAMOTORS', side='BUY', amount=1051.4, quantity=14.0, price=75.1, expiry_date=None)]]


In [ ]:
class Stock(BaseModel):
    stock_name: str
    expiry_date: Optional[datetime]
    
    def __init__(self,trade_record: TradeRecord):
        
        self.holding_quantity= 0.0
        self.avg_price = 0.0
        self.holding_amount = 0.0
        self.pnl_amount  = 0.0
        self.pnl_percentage = 0.0



    def trade(
        self, side: str, traded_price: float, traded_quantity: float
    ) -> Dict[str, Any]:
        """
        Executes a trade for the stock and updates its state.

        Args:
            side (str): The side of the trade, either 'BUY' or 'SELL'.
            traded_price (float): The price at which the stock was traded.
            traded_quantity (float): The quantity of the stock traded.

        Returns:
            dict: A dictionary containing details of the trade and updated stock state.
        """
        # BUY: positive position, SELL: negative position
        traded_quantity = (
            traded_quantity if side == "BUY" else (-1) * traded_quantity
        )

        if (self.holding_quantity * traded_quantity) >= 0:
            # Realized PnL
            pnl_amount = 0
            pnl_percentage = 0
            # Avg open price
            self.avg_price = (
                (self.avg_price * self.holding_quantity)
                + (traded_price * traded_quantity)
            ) / (self.holding_quantity + traded_quantity)
        else:
            # Calculate PnL and percentage
            pnl_amount = (
                (traded_price - self.avg_price)
                * min(abs(traded_quantity), abs(self.holding_quantity))
                * (abs(self.holding_quantity) / self.holding_quantity)
            )
            pnl_percentage = (
                pnl_amount
                / (
                    self.avg_price
                    * min(abs(traded_quantity), abs(self.holding_quantity))
                )
            ) * 100

            # Check if it is close-and-open
            if abs(traded_quantity) > abs(self.holding_quantity):
                self.avg_price = traded_price

        # Net position
        self.holding_quantity += traded_quantity
        self.side = side
        self.amount = abs(traded_price * traded_quantity)
        self.quantity = abs(traded_quantity)
        self.price = traded_price
        self.holding_amount = self.holding_quantity * self.avg_price
        self.pnl_amount = pnl_amount
        self.pnl_percentage = pnl_percentage

        return self.model_dump()

    def is_expired(self) -> bool:
        """
        Checks if a expiry_date is in the past.

        Returns:
            bool: True if the date is in the past, False otherwise.
        """

        return (
            datetime.today() > self.expiry_date
            if self.expiry_date is not None
            else False
        )

In [8]:
# Create a TradeRecord instance using keyword arguments by unpacking the dictionary
trade_record = TradeRecord(**df_trade_history.iloc[0])

print(trade_record)

datetime=Timestamp('2020-04-21 14:41:30') exchange='NSE' segment='EQ' stock_name='TATAMOTORS' side='BUY' amount=1051.4 quantity=14.0 price=75.1 expiry_date=None


In [ ]:
# Apply the trade logic to each row of the DataFrame
# Instantiate the Portfolio object
portfolio = Portfolio()

for trade_record in df_trade_history.astype(str).to_dict(orient="records"):
    portfolio.trade(trade_record)

portfolio.check_expired_stocks()

# Create a DataFrame from the processed data
df_trade_history = pd.DataFrame(data=portfolio.trade_history())

# Round the values in the columns to two decimal places
df_trade_history = df_trade_history.round(2)

# # Data cleanup
# df_trade_history = df_trade_history.replace("nan", None)
# df_trade_history = df_trade_history.fillna(None)

### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [ ]:
# # Save the result as a CSV file
# # Sort the DataFrame by 'segment', 'stock_name', and 'datetime'
# df_trade_history = df_trade_history.sort_values(
#     by=["segment", "stock_name", "datetime"]
# )
# # Select relevant columns
# df_trade_history = df_trade_history[
#     [
#         "datetime",
#         "exchange",
#         "segment",
#         "stock_name",
#         "scrip_code",
#         "expiry_date",
#         "side",
#         "quantity",
#         "price",
#         "amount",
#         "holding_quantity",
#         "avg_price",
#         "holding_amount",
#         "pnl_amount",
#         "pnl_percentage",
#     ]
# ]

# df_trade_history.to_csv(global_path.tradehistory_gold_file_path, index=None)
# logger.info("GOLD Layer CSV file for trade history successfully created at:")
# logger.info(global_path.tradehistory_gold_file_path.resolve())

# # Log the DataFrame info
# df_trade_history.info()